In [2]:
import pandas as pd 

In [24]:
df = pd.read_parquet("/a/scratch/alexm/research/mostrecent/gdb9_sto3g_converged.parquet")

In [7]:
df 

,smile,atom_positions,hlgap
0,NC=C(C=CC=O)C#N,"[-4.6443071378757415, -0.20404282270892277, -3...",2.140608
1,NCC#Cc1c[nH]nn1,"[6.0551210867647605, 1.231421557468469, 1.3027...",6.057728
2,N=C1CC(NC=O)=NN1,"[-5.575746050475118, 0.10778378863930606, -1.4...",5.244358
3,NC1C=C(F)C(F)C1F,"[3.9393057519100565, 0.04158247251484597, 0.56...",6.501084
4,CON=CC(C=O)=NO,"[5.852566316629086, -0.13053310990125, -0.2363...",3.952413
...,...,...,...
443475,NC1CC=CC2CC12N,"[-2.2813424026980695, -3.4155161626061767, 0.2...",6.341937
443476,CCC1N=CNC12CC2,"[5.164756344884526, -0.10817491930027225, -0.4...",6.640875
443477,CC(N)C(O)C(F)CO,"[-4.032985048718895, -1.5481892939251802, -1.8...",10.226885
443478,CN1CC1CC(O)CF,"[4.411501023747726, 1.032034092043031, 2.00537...",9.745535


In [41]:
import unicodedata
import numpy as np 
from tqdm.notebook import tqdm 
smiles = [np.fromstring(df["smile"].values[i], dtype=np.float64) for i in range(df.shape[0])]

/tmp/ipykernel_3046511/2320012492.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  smiles = [np.fromstring(df["smile"].values[i], dtype=np.float64) for i in range(df.shape[0])]


ValueError: string size must be a multiple of element size

In [49]:
#atom_positions = [np.fromstring(df["atom_positions"].values[i][1:-1], sep=",", dtype=np.float64) for i in tqdm(range(df.shape[0]))] # 192mb
# what if we pad and store as float32? 
# 

atom_positions = [np.fromstring(df["atom_positions"].values[i][1:-1], sep=",", dtype=np.float64).astype(np.float32) for i in tqdm(range(df.shape[0]))]

  0%|          | 0/416947 [00:00<?, ?it/s]

In [43]:
import awkward as ak 

In [44]:
atom_positions

[array([-4.64430714, -0.20404282, -3.52731689, -2.70660847,  0.010974  ,
        -1.97387504, -0.84759574,  0.2165922 , -0.48372155,  1.15294325,
         0.43791994,  1.11989442,  2.7199706 , -1.57842826,  1.56919911,
         4.80863832, -1.32557908,  3.24660636,  6.32206187, -3.09975474,
         3.74967771, -1.30357365,  3.55116964, -0.71254231, -1.59014953,
         5.78683378, -0.83622618, -4.43422991,  0.25906218, -5.48667044,
        -6.51879383, -0.80367568, -3.13024615, -2.19352346, -2.59095489,
        -2.26225426,  1.80022339,  2.10165692,  2.21397796,  2.31778607,
        -3.28884073,  0.59200639,  5.11715823,  0.52706755,  4.1559819 ]),
 array([ 6.05512109,  1.23142156,  1.30275411,  4.68231611, -1.09058107,
         1.52077933,  1.9529209 , -0.62423099,  0.90641706, -0.41407333,
        -0.24087522,  0.34117659, -2.93936299,  0.20168017, -0.23011751,
        -3.91805406, -0.44478528, -2.54093808, -6.32851321,  0.30124779,
        -2.65188497, -6.87555289,  1.44303355, -0

In [ ]:
ak_atom_positions = ak.Array(atom_positions)

In [46]:
print(ak_atom_positions)

[[-4.64, -0.204, -3.53, -2.71, 0.011, ..., 0.592, 5.12, 0.527, 4.16], ...]


In [50]:
print(ak_atom_positions.nbytes) #  192mb f64; so f16 would be 192/4 ~ 48 mb ;; similarly the 20GB becomes ~ 5 GB. 
# still very large; I wonder if there's more we can do to compress them atom positions? 
# perhps rdkit recomputation is the best idea? 

ak.to_parquet(ak_atom_positions, "test.parquet") 

190480168


  created_by: parquet-cpp-arrow version 11.0.0
  num_columns: 1
  num_rows: 416947
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 0

In [37]:
load = ak.from_parquet("test.parquet")

In [38]:
load

<Array [[], [6], [], [3], ..., [5], [], [4], []] type='416947 * var * int64'>

In [29]:
import pickle 
pickle.dump(atom_positions, open("test.pickle", "wb"))

In [ ]:
atom_positions

In [11]:
df.to_parquet("/a/scratch/alexm/research/mostrecent/compressed/ascii_version.parquet", compression="snappy")

In [13]:
df["smile"].values

array(['NC=C(C=CC=O)C#N', 'NCC#Cc1c[nH]nn1', 'N=C1CC(NC=O)=NN1', ...,
       'CC(N)C(O)C(F)CO', 'CN1CC1CC(O)CF', 'COCC(O)CCC=O'], dtype=object)